#CATALOG
## Metastore->Catalog->Schema->[Table,View,Volume,Functions/Models]
#####https://docs.databricks.com/aws/en/database-objects/

![](/Volumes/datamaster/bronze/images/metastore.png)

In [0]:
%sql
USE catalog datamaster
-- the above command will store or allow the following functions to be craeted under the datamaster catalog

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS bronze;
USE bronze;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS datamaster.bronze.test (
  id BIGINT PRIMARY KEY  GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1), -- This will generate a unique ID for each record
  name STRING,
  age INT,
  gender STRING
);

-- Inserting data into the table
-- insert data if it does not exist
INSERT INTO datamaster.bronze.test (name, age, gender)
SELECT *
FROM (
  VALUES 
    ('John', 25, 'Male'),
    ('Jane', 30, 'Female'),
    ('Bob', 40, 'Male'),
    ('Alice', 35, 'Female')
) AS v(name, age, gender)
WHERE NOT EXISTS (
  SELECT 1
  FROM datamaster.bronze.test t
  WHERE t.name = v.name 
    AND t.age = v.age
    AND t.gender = v.gender
);

    


In [0]:
%sql
SELECT * FROM datamaster.bronze.test LIMIT 10;

In [0]:
%sql
--DROP TABLE IF EXISTS datamaster.bronze.test;


##FUNCTIONS

In [0]:
%sql
CREATE TABLE IF NOT EXISTS datamaster.bronze.customer_reviews
(
customer_id INT,
first_name STRING,
last_name STRING,
review STRING,
rating INT
);
    
MERGE INTO datamaster.bronze.customer_reviews AS t
USING (
  SELECT * FROM VALUES
    (1, 'John',  'Doe',      'Great product!',   5),
    (2, 'Jane',  'Smith',    'Not so great',     2),
    (3, 'Bob',   'Johnson',  'Excellent',        5),
    (4, 'Alice', 'Williams', 'Very good',        4)
  AS v(customer_id, first_name, last_name, review, rating)
) AS v
ON t.customer_id = v.customer_id
WHEN NOT MATCHED THEN
  INSERT (customer_id, first_name, last_name, review, rating)
  VALUES (v.customer_id, v.first_name, v.last_name, v.review, v.rating);

    


In [0]:
%sql
SELECT * FROM datamaster.bronze.customer_reviews; 


In [0]:
%sql
-- CREATE FUNCTION function_name(Parameter datatype)
-- RETURNS datatype
-- RETURN expression/Logic;
    
-- CREATE FUNCTION add(x INT, y INT)
-- RETURNS INT
-- RETURN x + y;


In [0]:
%sql
CREATE FUNCTION IF NOT EXISTS datamaster.bronze.full_name_udf(first_name STRING, last_name STRING)
RETURNS STRING
RETURN CONCAT(first_name, ' ', last_name);

In [0]:
%sql
SELECT customer_id, datamaster.bronze.full_name_udf(first_name, last_name) AS full_name, review, rating FROM datamaster.bronze.customer_reviews;
    


In [0]:
%sql
-- create a complex UDF for sentiment Analysis
CREATE FUNCTION IF NOT EXISTS datamaster.bronze.sentiment_analysis(review STRING)
RETURNS STRING
RETURN 
    CASE
        WHEN review LIKE '%great%' OR review LIKE 'Great%' OR review LIKE '%excellent%' OR review LIKE 'Excellent%' OR review LIKE '%good%' THEN 'Positive'
        WHEN review LIKE '%not%' OR review LIKE '%bad%' OR review LIKE 'bad%' OR review LIKE '%poor%' OR review LIKE 'poor%' OR review LIKE '%worst%' OR review LIKE '%terrible%' THEN 'Negative'
        ELSE 'Neutral'
    END;
    


In [0]:
%sql
SELECT customer_id, datamaster.bronze.full_name_udf(first_name, last_name) AS full_name, review, datamaster.bronze.sentiment_analysis(review) AS sentiment, rating FROM datamaster.bronze.customer_reviews;

In [0]:
%sql
--DROP FUNCTION datamaster.bronze.sentiment_analysis